In [ ]:
import torch
import numpy as np
from datasets import Dataset
from tqdm.auto import tqdm
import os
import pickle
import pyphen
import conllu 
from transformers import (
    DistilBertTokenizerFast, 
    DistilBertForSequenceClassification, 
    DistilBertModel
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

print(" Step 1: Data Preparation for Probing Fine tuned model")

file_path = "en_ewt-ud-train.conllu"

try:
    print(f"Loading data from local file: {file_path}...")
    with open(file_path, "r", encoding="utf-8") as f:
        parsed_data = conllu.parse(f.read())
    
    words = []
    char_labels_data = []
    syll_labels_data = []

    char_label_map = {0: 'short', 1: 'medium', 2: 'long'}
    syll_label_map = {0: 'short', 1: 'medium', 2: 'long'}
    
    dic = pyphen.Pyphen(lang='en_US')

    def create_labels(token_list, feature_type='chars'):
        labels = []
        for token in token_list:
            if feature_type == 'chars':
                length = len(token)
                if length <= 4: labels.append(0)
                elif length <= 8: labels.append(1)
                else: labels.append(2)
            elif feature_type == 'syllables':
                syllables = dic.inserted(token.lower()).count('-') + 1 if token.strip() else 0
                if syllables <= 1: labels.append(0)
                elif syllables <= 2: labels.append(1)
                else: labels.append(2)
        return labels

    for sentence in tqdm(parsed_data, desc="Processing sentences"):
        tokens = [token['form'] for token in sentence]
        words.append(tokens)
        char_labels_data.append(create_labels(tokens, 'chars'))
        syll_labels_data.append(create_labels(tokens, 'syllables'))

    probing_dataset = Dataset.from_dict({
        'tokens': words,
        'char_labels': char_labels_data,
        'syll_labels': syll_labels_data
    })
    
    print("Dataset prepared for probing.")
    print(f"Number of sentences in dataset: {len(probing_dataset)}")
    
    SUBSET_SIZE = 2000
    if len(probing_dataset) > SUBSET_SIZE:
        probing_dataset = probing_dataset.shuffle(seed=42).select(range(SUBSET_SIZE))
    
    print(f"Total sentences in probing subset: {len(probing_dataset)}")
    print(f"Sample data: {probing_dataset[0]}")


except Exception as e:
    print(f"Error preparing dataset: {e}")
    exit()




 Step 1: Data Preparation for Probing Fine tuned model
Loading data from local file: en_ewt-ud-train.conllu...


Processing sentences: 100%|██████████| 12544/12544 [00:00<00:00, 58635.77it/s]


Dataset prepared for probing.
Number of sentences in dataset: 12544
Total sentences in probing subset: 2000
Sample data: {'tokens': ['A', 'key', 'question', 'is', 'how', 'they', 'acquired', 'the', 'anthrax', 'strain', 'first', 'isolated', 'by', 'the', 'Texas', 'Veterinary', 'Medical', 'Diagnostic', 'Lab', 'in', '1980', '.'], 'char_labels': [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0], 'syll_labels': [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0]}


In [ ]:
print("\n Step 2: Probing Setup ")

model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_DIR_FINE_TUNED = "/home/sharmajidotdev/manish/models/readability"
PROBE_RESULTS_DIR = "/home/sharmajidotdev/manish/probing_results"
os.makedirs(PROBE_RESULTS_DIR, exist_ok=True)


# probing function 
def get_word_level_embeddings_single_sentence(sentence_tokens, labels_for_sentence, tokenizer, model, device):
    base_bert_model = model.distilbert
    base_bert_model.eval()

    encoded_inputs = tokenizer(sentence_tokens, is_split_into_words=True, return_tensors='pt', padding=True, truncation=True, return_offsets_mapping=True, max_length=256)
    input_ids, attention_mask = encoded_inputs['input_ids'].to(device), encoded_inputs['attention_mask'].to(device)
    word_ids_list = encoded_inputs.word_ids()

    if not isinstance(word_ids_list, list) or not word_ids_list or not labels_for_sentence: return None, None
    
    with torch.no_grad():
        outputs = base_bert_model(input_ids=input_ids, attention_mask=attention_mask)
    hidden_states = outputs.hidden_states
    sentence_hidden_states_all_layers = [hs[0] for hs in hidden_states]
    
    word_to_subword_indices = {}
    for token_idx, word_idx in enumerate(word_ids_list):
        if word_idx is None or token_idx >= attention_mask.shape[1] or attention_mask[0, token_idx].item() == 0 or word_idx < 0: continue
        if word_idx not in word_to_subword_indices: word_to_subword_indices[word_idx] = []
        word_to_subword_indices[word_idx].append(token_idx)
    if not word_to_subword_indices: return None, None
    sorted_word_indices = sorted(word_to_subword_indices.keys())
    if len(sorted_word_indices) != len(labels_for_sentence): return None, None
    
    current_sentence_word_embeddings_by_layer = [[] for _ in range(7)]
    aligned_labels_for_sentence_output = []
    
    for original_word_idx in sorted_word_indices:
        subword_token_indices = word_to_subword_indices[original_word_idx]
        if not subword_token_indices: continue
        for layer_idx in range(7):
            if any(idx >= sentence_hidden_states_all_layers[layer_idx].shape[0] for idx in subword_token_indices): return None, None
            subword_embs = sentence_hidden_states_all_layers[layer_idx][subword_token_indices, :]
            current_sentence_word_embeddings_by_layer[layer_idx].append(subword_embs.mean(dim=0).cpu().numpy())
        aligned_labels_for_sentence_output.append(labels_for_sentence[original_word_idx])
    
    processed_word_embeddings_tensors = []
    for layer_idx in range(7):
        if current_sentence_word_embeddings_by_layer[layer_idx]:
            processed_word_embeddings_tensors.append(torch.from_numpy(np.vstack(current_sentence_word_embeddings_by_layer[layer_idx])).float())
        else: return None, None
    return processed_word_embeddings_tensors, aligned_labels_for_sentence_output





 Step 2: Probing Setup 
Using device: cuda


In [ ]:
print("\n Step 3: Probing Fine tuned models ")

probe_results_char_ft = {f"layer_{i}": {'accuracy': [], 'f1': []} for i in range(7)}
probe_results_syll_ft = {f"layer_{i}": {'accuracy': [], 'f1': []} for i in range(7)}

N_SPLITS = 5
for fold_idx in range(N_SPLITS):
    print(f"\n Probing Fine-Tuned Model from Fold {fold_idx + 1} ")

    # loading fine-tuned models for FKGL and FRE for this fold
    ft_model_fkgl = DistilBertForSequenceClassification.from_pretrained(
        os.path.join(MODEL_DIR_FINE_TUNED, "Fkgl", f"best_fold_{fold_idx}"),
        output_hidden_states=True
    ).to(device)
    
    ft_model_fre = DistilBertForSequenceClassification.from_pretrained(
        os.path.join(MODEL_DIR_FINE_TUNED, "Fre", f"best_fold_{fold_idx}"),
        output_hidden_states=True
    ).to(device)

    #  probing for NCHAR on FKGL-tuned model 
    print("\nExtracting embeddings from FKGL-tuned model and training probes for NCHAR...")
    all_embs_nchar_ft = [[] for _ in range(7)]
    all_labels_nchar_ft = []
    for i in tqdm(range(len(probing_dataset['tokens'])), desc="NCHAR Probing"):
        embs, labels = get_word_level_embeddings_single_sentence(probing_dataset['tokens'][i], probing_dataset['char_labels'][i], tokenizer, ft_model_fkgl, device)
        if embs:
            for l_idx, e_list in enumerate(embs):
                all_embs_nchar_ft[l_idx].append(e_list)
            all_labels_nchar_ft.extend(labels)
    
    concatenated_embs_nchar_ft = [torch.cat(embs, dim=0) for embs in all_embs_nchar_ft]
    y_nchar_ft = np.array(all_labels_nchar_ft)

    for l_idx in range(7):
        X_train, X_val, y_train, y_val = train_test_split(concatenated_embs_nchar_ft[l_idx].cpu().numpy(), y_nchar_ft, test_size=0.2, stratify=y_nchar_ft)
        probe = LogisticRegression(max_iter=1000, multi_class='multinomial', n_jobs=-1).fit(X_train, y_train)
        y_pred = probe.predict(X_val)
        probe_results_char_ft[f"layer_{l_idx}"]['accuracy'].append(accuracy_score(y_val, y_pred))
        probe_results_char_ft[f"layer_{l_idx}"]['f1'].append(f1_score(y_val, y_pred, average='weighted'))

    # probing for NSYLL on FRE-tuned model 
    print("\nExtracting embeddings from FRE-tuned model and training probes for NSYLL...")
    all_embs_nsyll_ft = [[] for _ in range(7)]
    all_labels_nsyll_ft = []
    for i in tqdm(range(len(probing_dataset['tokens'])), desc="NSYLL Probing"):
        embs, labels = get_word_level_embeddings_single_sentence(probing_dataset['tokens'][i], probing_dataset['syll_labels'][i], tokenizer, ft_model_fre, device)
        if embs:
            for l_idx, e_list in enumerate(embs):
                all_embs_nsyll_ft[l_idx].append(e_list)
            all_labels_nsyll_ft.extend(labels)
    
    concatenated_embs_nsyll_ft = [torch.cat(embs, dim=0) for embs in all_embs_nsyll_ft]
    y_nsyll_ft = np.array(all_labels_nsyll_ft)

    for l_idx in range(7):
        X_train, X_val, y_train, y_val = train_test_split(concatenated_embs_nsyll_ft[l_idx].cpu().numpy(), y_nsyll_ft, test_size=0.2, stratify=y_nsyll_ft)
        probe = LogisticRegression(max_iter=1000, multi_class='multinomial', n_jobs=-1).fit(X_train, y_train)
        y_pred = probe.predict(X_val)
        probe_results_syll_ft[f"layer_{l_idx}"]['accuracy'].append(accuracy_score(y_val, y_pred))
        probe_results_syll_ft[f"layer_{l_idx}"]['f1'].append(f1_score(y_val, y_pred, average='weighted'))
    
    # saving probing results for this fold
    current_fold_results = {
        'nchar': probe_results_char_ft,
        'nsyll': probe_results_syll_ft
    }
    results_file_path = os.path.join(PROBE_RESULTS_DIR, f"fold_{fold_idx}_probing_results.pkl")
    with open(results_file_path, 'wb') as f:
        pickle.dump(current_fold_results, f)
    print(f"Saved probing results for Fold {fold_idx+1} to {results_file_path}")




 Step 3: Probing Fine tuned models 

 Probing Fine-Tuned Model from Fold 1 

Extracting embeddings from FKGL-tuned model and training probes for NCHAR...


NCHAR Probing: 100%|██████████| 2000/2000 [00:17<00:00, 112.24it/s]
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sh


Extracting embeddings from FRE-tuned model and training probes for NSYLL...


NSYLL Probing: 100%|██████████| 2000/2000 [00:16<00:00, 118.66it/s]
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sh

Saved probing results for Fold 1 to /home/sharmajidotdev/manish/probing_results/fold_0_probing_results.pkl

 Probing Fine-Tuned Model from Fold 2 

Extracting embeddings from FKGL-tuned model and training probes for NCHAR...


...

Saved probing results for Fold 4 to /home/sharmajidotdev/manish/probing_results/fold_3_probing_results.pkl

 Probing Fine-Tuned Model from Fold 5 

Extracting embeddings from FKGL-tuned model and training probes for NCHAR...


NCHAR Probing: 100%|██████████| 2000/2000 [00:16<00:00, 118.17it/s]
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sh


Extracting embeddings from FRE-tuned model and training probes for NSYLL...


NSYLL Probing: 100%|██████████| 2000/2000 [00:17<00:00, 117.34it/s]
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sharmajidotdev/manish/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/sh

Saved probing results for Fold 5 to /home/sharmajidotdev/manish/probing_results/fold_4_probing_results.pkl


In [10]:
print("\n Step 4: Final Summary ")
avg_probe_acc_char_ft = [np.mean(probe_results_char_ft[f"layer_{i}"]['accuracy']) for i in range(7)]
avg_probe_f1_char_ft = [np.mean(probe_results_char_ft[f"layer_{i}"]['f1']) for i in range(7)]
avg_probe_acc_syll_ft = [np.mean(probe_results_syll_ft[f"layer_{i}"]['accuracy']) for i in range(7)]
avg_probe_f1_syll_ft = [np.mean(probe_results_syll_ft[f"layer_{i}"]['f1']) for i in range(7)]

print("Final Probing Results on Fine-Tuned Models (Average across 5 folds)")
print("------------------------------------------------------------------")
print("Number of Characters (NCHAR) Probe Accuracy:", avg_probe_acc_char_ft)
print("Number of Characters (NCHAR) Probe F1-Score:", avg_probe_f1_char_ft)
print("\nNumber of Syllables (NSYLL) Probe Accuracy:", avg_probe_acc_syll_ft)
print("Number of Syllables (NSYLL) Probe F1-Score:", avg_probe_f1_syll_ft)


 Step 4: Final Summary 
Final Probing Results on Fine-Tuned Models (Average across 5 folds)
------------------------------------------------------------------
Number of Characters (NCHAR) Probe Accuracy: [np.float64(0.9429654335961188), np.float64(0.9330806549423893), np.float64(0.9196179502728926), np.float64(0.9050939963614312), np.float64(0.8928744693753791), np.float64(0.880230442692541), np.float64(0.8697392359005459)]
Number of Characters (NCHAR) Probe F1-Score: [np.float64(0.9422132907847134), np.float64(0.9325676842977482), np.float64(0.9193849328727989), np.float64(0.904227673252182), np.float64(0.8920164315899548), np.float64(0.8789594188774214), np.float64(0.8679934563338982)]

Number of Syllables (NSYLL) Probe Accuracy: [np.float64(0.9339902971497878), np.float64(0.9219526986052152), np.float64(0.9093086719223772), np.float64(0.9016373559733172), np.float64(0.8901152213462705), np.float64(0.876773802304427), np.float64(0.874711946634324)]
Number of Syllables (NSYLL) Probe 

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

RESULTS_DIR = "/home/sharmajidotdev/manish/probing_results"
OUTPUT_CSV_PATH = "/home/sharmajidotdev/manish/probing_results/final_probing_results.csv"

all_results = []

#  dictionaries to store results across all folds
results_nchar = {f"layer_{i}": {'accuracy': [], 'f1': []} for i in range(7)}
results_nsyll = {f"layer_{i}": {'accuracy': [], 'f1': []} for i in range(7)}

for fold_idx in range(5):
    results_file_path = os.path.join(PROBE_RESULTS_DIR, f"fold_{fold_idx}_probing_results.pkl")
    
    try:
        with open(results_file_path, 'rb') as f:
            loaded_results = pickle.load(f)
            
            nchar_results_fold = loaded_results['nchar']
            nsyll_results_fold = loaded_results['nsyll']
            
            for layer_key, metrics in nchar_results_fold.items():
                results_nchar[layer_key]['accuracy'].extend(metrics['accuracy'])
                results_nchar[layer_key]['f1'].extend(metrics['f1'])

            for layer_key, metrics in nsyll_results_fold.items():
                results_nsyll[layer_key]['accuracy'].extend(metrics['accuracy'])
                results_nsyll[layer_key]['f1'].extend(metrics['f1'])

    except FileNotFoundError:
        print(f"Warning: Results file for fold {fold_idx+1} not found. Skipping.")
        
final_records = []

# processing NCHAR results
for layer_idx in range(7):
    layer_key = f"layer_{layer_idx}"
    avg_acc = np.mean(results_nchar[layer_key]['accuracy'])
    avg_f1 = np.mean(results_nchar[layer_key]['f1'])
    final_records.append({'feature': 'NCHAR', 'layer': layer_idx, 'accuracy': avg_acc, 'f1': avg_f1})
    
# processing NSYLL results
for layer_idx in range(7):
    layer_key = f"layer_{layer_idx}"
    avg_acc = np.mean(results_nsyll[layer_key]['accuracy'])
    avg_f1 = np.mean(results_nsyll[layer_key]['f1'])
    final_records.append({'feature': 'NSYLL', 'layer': layer_idx, 'accuracy': avg_acc, 'f1': avg_f1})

results_df = pd.DataFrame(final_records)

# saving  dataframe to CSV 
results_df.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"\nSuccessfully aggregated and saved final probing results to '{OUTPUT_CSV_PATH}'.")
print("\nFinal aggregated results:")
print(results_df)



Successfully aggregated and saved final probing results to '/home/sharmajidotdev/manish/probing_results/final_probing_results.csv'.

Final aggregated results:
   feature  layer  accuracy        f1
0    NCHAR      0  0.942399  0.941574
1    NCHAR      1  0.932282  0.931789
2    NCHAR      2  0.920740  0.920543
3    NCHAR      3  0.903881  0.903061
4    NCHAR      4  0.892137  0.891292
5    NCHAR      5  0.881625  0.880414
6    NCHAR      6  0.868941  0.867051
7    NSYLL      0  0.933808  0.933462
8    NSYLL      1  0.921720  0.921463
9    NSYLL      2  0.909430  0.908869
10   NSYLL      3  0.901334  0.900921
11   NSYLL      4  0.890075  0.889426
12   NSYLL      5  0.876087  0.874680
13   NSYLL      6  0.874207  0.872226


In [ ]:
import pickle
import os

file_path = os.path.join("probing_results/fold_0_probing_results.pkl") 

try:
    with open(file_path, 'rb') as f:
        loaded_results = pickle.load(f)

    print("Successfully loaded results from the pickle file.")
    print(loaded_results)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading pickle file: {e}")

Successfully loaded results from the pickle file.
{'nchar': {'layer_0': {'accuracy': [0.9416312916919345], 'f1': [0.9408322353600702]}, 'layer_1': {'accuracy': [0.9307155852031535], 'f1': [0.9305592217162694]}, 'layer_2': {'accuracy': [0.9217707701637355], 'f1': [0.9218651595516834]}, 'layer_3': {'accuracy': [0.9010006064281383], 'f1': [0.9002654492524436]}, 'layer_4': {'accuracy': [0.8893268647665251], 'f1': [0.8886854566946439]}, 'layer_5': {'accuracy': [0.886294724075197], 'f1': [0.8848913181373872]}, 'layer_6': {'accuracy': [0.8673438447543966], 'f1': [0.8652369576201351]}}, 'nsyll': {'layer_0': {'accuracy': [0.9334445118253487], 'f1': [0.9331247530466945]}, 'layer_1': {'accuracy': [0.9225288053365677], 'f1': [0.922243583568802]}, 'layer_2': {'accuracy': [0.9096422073984233], 'f1': [0.9087742302785121]}, 'layer_3': {'accuracy': [0.8979684657368102], 'f1': [0.8975279297970576]}, 'layer_4': {'accuracy': [0.8887204366282595], 'f1': [0.8881481942534566]}, 'layer_5': {'accuracy': [0.875

In [ ]:
import pickle
import os

file_path = os.path.join("probing_results/fold_1_probing_results.pkl") 

try:
    with open(file_path, 'rb') as f:
        loaded_results = pickle.load(f)

    print("Successfully loaded results from the pickle file.")
    print(loaded_results)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading pickle file: {e}")

Successfully loaded results from the pickle file.
{'nchar': {'layer_0': {'accuracy': [0.9416312916919345, 0.9445118253486962], 'f1': [0.9408322353600702, 0.9437649025969688]}, 'layer_1': {'accuracy': [0.9307155852031535, 0.9287446937537902], 'f1': [0.9305592217162694, 0.9280776765916897]}, 'layer_2': {'accuracy': [0.9217707701637355, 0.9241964827167981], 'f1': [0.9218651595516834, 0.9236568864905152]}, 'layer_3': {'accuracy': [0.9010006064281383, 0.9038811400849], 'f1': [0.9002654492524436, 0.9031709480028262]}, 'layer_4': {'accuracy': [0.8893268647665251, 0.8928138265615525], 'f1': [0.8886854566946439, 0.8918492119171202]}, 'layer_5': {'accuracy': [0.886294724075197, 0.8788659793814433], 'f1': [0.8848913181373872, 0.878243292700821]}, 'layer_6': {'accuracy': [0.8673438447543966, 0.8681018799272286], 'f1': [0.8652369576201351, 0.8662995501524938]}}, 'nsyll': {'layer_0': {'accuracy': [0.9334445118253487, 0.9343541540327471], 'f1': [0.9331247530466945, 0.9339967704756217]}, 'layer_1': {'

In [ ]:
import pickle
import os

file_path = os.path.join("probing_results/fold_2_probing_results.pkl") 

try:
    with open(file_path, 'rb') as f:
        loaded_results = pickle.load(f)

    print("Successfully loaded results from the pickle file.")
    print(loaded_results)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading pickle file: {e}")

Successfully loaded results from the pickle file.
{'nchar': {'layer_0': {'accuracy': [0.9416312916919345, 0.9445118253486962, 0.9387507580351728], 'f1': [0.9408322353600702, 0.9437649025969688, 0.9374751343663253]}, 'layer_1': {'accuracy': [0.9307155852031535, 0.9287446937537902, 0.9364766525166768], 'f1': [0.9305592217162694, 0.9280776765916897, 0.9357339304912485]}, 'layer_2': {'accuracy': [0.9217707701637355, 0.9241964827167981, 0.9197998787143723], 'f1': [0.9218651595516834, 0.9236568864905152, 0.9196006094421237]}, 'layer_3': {'accuracy': [0.9010006064281383, 0.9038811400849, 0.9044875682231656], 'f1': [0.9002654492524436, 0.9031709480028262, 0.9035549008356476]}, 'layer_4': {'accuracy': [0.8893268647665251, 0.8928138265615525, 0.8938750758035173], 'f1': [0.8886854566946439, 0.8918492119171202, 0.8931199594719471]}, 'layer_5': {'accuracy': [0.886294724075197, 0.8788659793814433, 0.8784111582777441], 'f1': [0.8848913181373872, 0.878243292700821, 0.8771290272477056]}, 'layer_6': {'a

In [ ]:
import pickle
import os

file_path = os.path.join("probing_results/fold_3_probing_results.pkl") 

try:
    with open(file_path, 'rb') as f:
        loaded_results = pickle.load(f)

    # printing  contents of  dictionary
    print("Successfully loaded results from the pickle file.")
    print(loaded_results)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading pickle file: {e}")

Successfully loaded results from the pickle file.
{'nchar': {'layer_0': {'accuracy': [0.9416312916919345, 0.9445118253486962, 0.9387507580351728, 0.9436021831412977], 'f1': [0.9408322353600702, 0.9437649025969688, 0.9374751343663253, 0.9429679840611409]}, 'layer_1': {'accuracy': [0.9307155852031535, 0.9287446937537902, 0.9364766525166768, 0.9367798665858096], 'f1': [0.9305592217162694, 0.9280776765916897, 0.9357339304912485, 0.9360585720311106]}, 'layer_2': {'accuracy': [0.9217707701637355, 0.9241964827167981, 0.9197998787143723, 0.9137355973317162], 'f1': [0.9218651595516834, 0.9236568864905152, 0.9196006094421237, 0.9135820371152626]}, 'layer_3': {'accuracy': [0.9010006064281383, 0.9038811400849, 0.9044875682231656, 0.9081261370527592], 'f1': [0.9002654492524436, 0.9031709480028262, 0.9035549008356476, 0.9070907098482137]}, 'layer_4': {'accuracy': [0.8893268647665251, 0.8928138265615525, 0.8938750758035173, 0.8941782898726501], 'f1': [0.8886854566946439, 0.8918492119171202, 0.8931199

In [ ]:
import pickle
import os

file_path = os.path.join("probing_results/fold_4_probing_results.pkl") 

try:
    with open(file_path, 'rb') as f:
        loaded_results = pickle.load(f)
    print("Successfully loaded results from the pickle file.")
    print(loaded_results)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading pickle file: {e}")

Successfully loaded results from the pickle file.
{'nchar': {'layer_0': {'accuracy': [0.9416312916919345, 0.9445118253486962, 0.9387507580351728, 0.9436021831412977, 0.946331109763493], 'f1': [0.9408322353600702, 0.9437649025969688, 0.9374751343663253, 0.9429679840611409, 0.9460261975390621]}, 'layer_1': {'accuracy': [0.9307155852031535, 0.9287446937537902, 0.9364766525166768, 0.9367798665858096, 0.9326864766525167], 'f1': [0.9305592217162694, 0.9280776765916897, 0.9357339304912485, 0.9360585720311106, 0.9324090206584222]}, 'layer_2': {'accuracy': [0.9217707701637355, 0.9241964827167981, 0.9197998787143723, 0.9137355973317162, 0.9185870224378411], 'f1': [0.9218651595516834, 0.9236568864905152, 0.9196006094421237, 0.9135820371152626, 0.9182199717644092]}, 'layer_3': {'accuracy': [0.9010006064281383, 0.9038811400849, 0.9044875682231656, 0.9081261370527592, 0.9079745300181928], 'f1': [0.9002654492524436, 0.9031709480028262, 0.9035549008356476, 0.9070907098482137, 0.9070563583217789]}, 'la